# Task 3
## Quantum Compiler 

Here, I have a built a basic working quantum compiler that can successfully build an n-qubit quantum circuit and apply various gates on it. The currently supported gates are the pauli gates (X, Y, Z), Hadamard, S, and T. This compiler also supports multi-controlled versions of these gates.

I have used numpy to simulate the statevectors and gates, and have considered every circuit to be an object of the qckt class. Thus, you can work with multiple circuits at the same time.

The final measurement is done with the random.choice function of numpy, using weighted probablities.

In [1]:
import numpy as np

In [2]:
# Defining all gates

# Identity gate
I = np.identity(2)

# X gate 
X = np.array([
[0, 1], 
[1, 0]
])

# Y gate
Y = np.array([
[0, -1.0j], 
[1.0j, 0]
])

# Z gate
Z = np.array([
[1, 0], 
[0, -1]
])

# H gate
H = np.array([
[1/np.sqrt(2), 1/np.sqrt(2)],
[1/np.sqrt(2), -1/np.sqrt(2)]
])

# S gate
S = np.array([
[1, 0],
[0, 1.0j]
])

# T gate
T = np.array([
[1, 0],
[0, np.e**((np.pi)*1j*(1/4))]
])

gate_map = {
    'I' : I,
    'X' : X,
    'Y' : Y,
    'Z' : Z,
    'H' : H,
    'S' : S,
    'T' : T
}

zero_x_ket = np.array([[1], [0]])
one_x_ket = np.array([[0], [1]])

zero_x_bra = np.swapaxes(zero_x_ket.conjugate(), 0, 1)
one_x_bra = np.swapaxes(one_x_ket.conjugate(), 0, 1)

proj_0x0 = np.matmul(zero_x_ket, zero_x_bra)
proj_1x1 = np.matmul(one_x_ket, one_x_bra)

In [40]:
class qckt:
    
    # circuit initialization 
    def __init__(self, no_of_qubits):  
        self.nq = no_of_qubits
        self.state = np.zeros(2**no_of_qubits)
        self.state[0] = 1
        
    # builds gate to be applied to the circuit
    def get_operator(self, gate, target, control = [-1]):
        
        # non-controlled operation
        if control[0] == -1:
            op = np.array([[1]])
            for i in range(self.nq):
                if i == target:
                    op = np.kron(op, gate)
                else:
                    op = np.kron(op, I)
        
        # controlled operation 
        # this has been calculated as per the formula given in the material
        else:
        
            control.sort() 
            curr = 0             
            op = np.array([[1]])
            for j in range(self.nq):                
                if j == control[curr]:
                    op = np.kron(op, proj_1x1)
                    curr += 1
                    if curr == len(control):
                        curr = 0
                elif j == target:
                    op = np.kron(op, gate)
                else:
                    op = np.kron(op, I)
            
            ctrls = {'0' : proj_0x0, '1' : proj_1x1}          
            
            for i in range(2**len(control)-1):
                
                part = np.array([[1]])
                part_num = bin(i)[2:].zfill(len(control))
                curr = 0
                
                for j in range(self.nq):
                    if j == control[curr]:
                        part = np.kron(part, ctrls[part_num[curr]])
                        curr += 1;
                        if curr == len(control):
                            curr = 0
                    else:
                        part = np.kron(part, I)
                op = np.add(op, part)
                
        return op   
    
    # apply the unitary to the current statevector
    def apply_gate(self, instr):
        k = len(instr[0])
        if k == 1:
            self.state = np.matmul(self.get_operator(gate_map[instr[0]], int(instr[1])), self.state)
        else:
            self.state = np.matmul(self.get_operator(gate_map[instr[0][-1]], int(instr[1]), [int(instr[i]) for i in range(2, k+1)]), self.state)
        print(self.state)
    
    # measure all the qubits and get the final state in little-endian notation
    def meas(self):
        all_states = [bin(i)[2:][::-1] for i in range(0,2**self.nq)]
        print("Measured state : ", np.random.choice(all_states, p = np.square(self.state)))
        
        

## Using the compiler

Finally! You can now use the compiler to build your own circuits.

Start by specifying the number of qubits you will be working with.

The circuit building is next.
You can construct their circuit by entering a series of instructions that will apply the operator on the specified qubit.

The following is the syntax:

```gate target_qubit (control_qubits)```

For example, to apply a Hadamard gate on the 1st qubit, write

```H 0```

To apply a CZ gate on the 1st qubit, keeping the 0th qubit as control:

```CZ 1 0```

To apply a CCX gate on the 2nd qubit, keeping the 0th and 1st qubits as control:

```CCX 2 0 1```

Case and spacing do not matter, it's handled :)

When you have finished building the circuit, enter 0.

In [37]:
print("Enter the number of qubits you want to work with : ")
n = int(input())
ckt = qckt(n)


print("Build your circuit here :")
command = input()
while command != "0" :
    instr = [i.strip().upper() for i in command.split()]
    ckt.apply_gate(instr)
    command = input()

Enter the number of qubits you want to work with : 
3
Build your circuit here :
X 0
[0. 0. 0. 0. 1. 0. 0. 0.]
CX 1 0
[0. 0. 0. 0. 0. 0. 1. 0.]
CCH 2 1 0
[0.         0.         0.         0.         0.         0.
 0.70710678 0.70710678]
0


## Measurement


In [41]:
ckt.meas()

Measured state :  011
